Processing the output files here,

In [2]:
import os
import pandas as pd
import numpy as np
import re
import json

In [3]:
def check_error_log(lfp0, lfp1, lpf2):
    paths = [lfp0, lfp1, lpf2 ]
    for log_file_path in paths:
            with open(log_file_path, 'r') as file:
                for line in file:
                    if len(line) >= 1 and ( line[0] == 'E' or line[0] == 'F' ):
                        extracted_string = line[0:5]
                        return extracted_string
    return None


In [4]:
def get_execution_time(lfp0, lfp1, lpf2 ): # ASSIGN FIRST AND LAST TIMESTAMP TO MIN AND MAX AMONG ALL THESE FILES
    paths = [lfp0, lfp1, lpf2 ]
    first_timestamp = None
    last_timestamp = None
    for log_file_path in paths:

        with open(log_file_path, 'r') as file:
            timestamp_pattern = re.compile(r'^[WIE](\d{4}) (\d{2}:\d{2}:\d{2}\.\d+)')
            for line in file:
                match = timestamp_pattern.search(line)
                if match:
                    time_str = match.group(2)
                    try:
                        timestamp = pd.to_datetime(time_str, format='%H:%M:%S.%f')
                        # Update the first and last timestamps
                        if first_timestamp is None:
                            first_timestamp = timestamp
                        if last_timestamp is None:
                            last_timestamp = timestamp
                        #not none
                        if first_timestamp > timestamp:
                            first_timestamp = timestamp #update to minimum
                        if last_timestamp < timestamp:
                            last_timestamp = timestamp #update to mmax
                    except ValueError as ve:
                        # Handle invalid timestamp formats
                        print(f"Skipping invalid timestamp: {timestamp_str} ({ve})")

            # invalid case
            if first_timestamp is None or last_timestamp is None:
                print("No valid timestamps found")
                return pd.Timedelta(0)
            execution_time = last_timestamp - first_timestamp
            return execution_time

In [5]:
def check_liveness(lfp0, lfp1, lpf2):
  paths = [lfp0, lfp1, lpf2]
  search_phrase = "Node becomes leader"
  for log_file_path in paths:
        with open(log_file_path, 'r') as file:
            for line in file:
                if search_phrase.lower() in line.lower():
                        return True
  return False

In [6]:
def check_safety(lfp0, lfp1, lpf2):
  paths = [lfp0, lfp1, lpf2]
  search_phrase = r"term (\d+) become leader of group" # Regex to match 'term 1', 'term 2', etc.

  terms = {}
  for path in paths:
      with open(path, 'r') as file:
          content = file.read()
          terms[path] = set(re.findall(search_phrase, content))

  # if termsA ^ termsB =! null -> TRUE
  for i, path1 in enumerate(paths):
      for j, path2 in enumerate(paths):
          if i != j:
              if terms[path1] & terms[path2]:  
                  return True
  return False

In [7]:
def parse_schedule_log(log_file_path):
    schedule_data = []
    with open(log_file_path, 'r') as file:
        for line in file:
            match_message= re.match(r"\{(\d+) (\d+) (\w+) SenderVC: (.+) ReceiverVC: (.+)\}", line.strip())
            match_drop= re.match(r"\{(\d+) (\d+) (\w+) is dropped SenderVC: (.+) ReceiverVC: (.+)\}", line.strip())

            if match_message:
                entry = {
                    'drop': 0,
                    'sender': int(match_message.group(1)),
                    'receiver': int(match_message.group(2)),
                    'action': match_message.group(3),
                    'sender_vc': match_message.group(4),
                    'receiver_vc': match_message.group(5)
                }
                schedule_data.append(entry)

            if match_drop:
                entry = {
                    'drop': 1,
                    'sender': int(match_drop.group(1)),
                    'receiver': int(match_drop.group(2)),
                    'action': match_drop.group(3),
                    'sender_vc': match_drop.group(4),
                    'receiver_vc': match_drop.group(5)
                }
                schedule_data.append(entry)
    return schedule_data

In [8]:

benchmarks = ["b0", "b1", "b2"]
schedulers = ["pct1",  "pct2", "pos", "posc", "random"]
dmFaults = ["f0", "f4"]

data = []

# Loop through each combination of benchmark, scheduler, and fault condition
for benchmark in benchmarks:
    for scheduler in schedulers:
        for faulter in dmFaults:
          
          
            for i in range(0, 100):
                if scheduler == "pct1" or scheduler ==  "pct2":
                    scheduler2 = scheduler [0:-1]
                elif scheduler == "posc":
                    scheduler2 = "posconflict"
                else:
                    scheduler2 = scheduler
                experiment_folder = os.path.join(f"{benchmark}-out", f"{benchmark}-{faulter}-{scheduler}", f"test_{scheduler2}_{i}")
                
               
                if not os.path.exists(experiment_folder):
                    print(f"Skipping non-existent folder: {experiment_folder}")
                    continue
                
                schedule_path = os.path.join(experiment_folder, "schedule.log")
                stderr0_path = os.path.join(experiment_folder, "stderr_0.log")
                stderr1_path = os.path.join(experiment_folder, "stderr_1.log")
                stderr2_path = os.path.join(experiment_folder, "stderr_2.log")
            
                parsed_schedule = parse_schedule_log(schedule_path)
                errorLog = check_error_log(stderr0_path, stderr1_path, stderr2_path)
                t_exec = get_execution_time(stderr0_path, stderr1_path, stderr2_path)
                livenessCheck = check_liveness(stderr0_path, stderr1_path, stderr2_path)
                safetyCheck = check_safety(stderr0_path, stderr1_path, stderr2_path)

                fault = False
                if faulter == "f4":
                    fault = True
                    
                # Create entry and append to data
                entry = {
                    'benchmark': benchmark,
                    'scheduler': scheduler,
                    'drop message faults': fault,
                    'Error log': errorLog,
                    'liveness': livenessCheck,
                    'safety violation': safetyCheck,
                    'Schedule': parsed_schedule,
                    'Execution_time': t_exec
                }
                data.append(entry)
               


In [9]:
df = pd.DataFrame(data)
df.head()

,benchmark,scheduler,drop message faults,Error log,liveness,safety violation,Schedule,Execution_time
0,b0,pct1,False,None,True,False,"[{'drop': 0, 'sender': 1, 'receiver': 2, 'acti...",0 days 00:00:16.801700
1,b0,pct1,False,None,False,False,"[{'drop': 0, 'sender': 1, 'receiver': 2, 'acti...",0 days 00:00:18.355514
2,b0,pct1,False,None,False,False,"[{'drop': 0, 'sender': 0, 'receiver': 2, 'acti...",0 days 00:00:17.525810
3,b0,pct1,False,None,True,False,"[{'drop': 0, 'sender': 2, 'receiver': 1, 'acti...",0 days 00:00:16.275403
4,b0,pct1,False,E0824,True,False,"[{'drop': 0, 'sender': 0, 'receiver': 2, 'acti...",0 days 00:00:17.790498


scheduler | fault condifiton | number of unique schedules | number of unique traces | number unieuq error lines | number of possible liveness bugs | number of safety violations | 

In [10]:
def serialize_schedule(schedule): #this way comparison will be faster (?)
    return json.dumps(schedule)

In [11]:
def get_trace(nodeID, schedule):
    filtered_schedule = [entry for entry in schedule if entry['sender'] == nodeID or entry['receiver'] == nodeID]
    return filtered_schedule

In [12]:

statistics = []

for benchmark in set(entry['benchmark'] for entry in data):
    for scheduler in set(entry['scheduler'] for entry in data):
        for fault in set(entry['drop message faults'] for entry in data):
            filtered_entries = [entry for entry in data 
                                if entry['benchmark'] == benchmark and
                                   entry['scheduler'] == scheduler and
                                   entry['drop message faults'] == fault]

            if not filtered_entries:
                continue

            filtered_entries_df = pd.DataFrame(filtered_entries)
            serialized_schedules= filtered_entries_df['Schedule'].apply(serialize_schedule)
            unique_schedules = serialized_schedules.unique()
            unique_schedules = len(unique_schedules)

            
            # Calculate average length of schedules
            schedule_lengths = filtered_entries_df['Schedule'].apply(len)
            avg_schedule_length = schedule_lengths.mean()
            
            tracesOfN0= filtered_entries_df['Schedule'].apply(lambda s: get_trace(0,s))
            tracesOfN1= filtered_entries_df['Schedule'].apply(lambda s: get_trace(1,s))
            tracesOfN2= filtered_entries_df['Schedule'].apply(lambda s: get_trace(2,s))
            #tra_n0_ser = tracesOfN0.apply(serialize_schedule)
            #tra_n1_ser = tracesOfN1.apply(serialize_schedule)
            #tra_n2_ser = tracesOfN2.apply(serialize_schedule)
            #traces: 
                # node 1: receive A from 0, send B to 2, receive B from 0
                # node 2: receive A from 0, send B to 1, receive B from 0

            general_traces = pd.concat([tracesOfN0, tracesOfN2, tracesOfN1])
            
            traces_ser = general_traces.apply(serialize_schedule)
            unique_traces = len(traces_ser.unique())
            #unique_error_lines = set(line for entry in filtered_entries if entry['Error log'] is not None for line in entry['Error log'])
  
            num_error_lines = filtered_entries_df['Error log'].notna().sum()

            filtered_entries_with_errors = filtered_entries_df[filtered_entries_df['Error log'].notna()]

            error_logs = filtered_entries_with_errors['Error log']

            error_logs_list = error_logs.tolist()
            error_logs_set = set(error_logs)
            num_unique_error_logs = len(error_logs_set)
            #print(error_logs_set)
            num_liveness_bugs = (filtered_entries_df['liveness'] == False).sum()
            
            num_safety_violations = (filtered_entries_df['safety violation'] == True).sum()

            mean_Texec = filtered_entries_df['Execution_time'].mean()
        
            statistics.append({
                'benchmark' : benchmark,
                'scheduler': scheduler,
                'fault condition': fault,
                'number of unique schedules': unique_schedules,
                'average schedule length': avg_schedule_length, 
                'number of unique traces': unique_traces,
                'number of tests with error or failure logged':num_error_lines,
                'unique errors or failures logged':error_logs_set,
                'number of possible liveness bugs': num_liveness_bugs,
                'number of safety violations': num_safety_violations,
                'mean execution time' : mean_Texec
            })

df_statistics = pd.DataFrame(statistics)

df_statistics


,benchmark,scheduler,fault condition,number of unique schedules,average schedule length,number of unique traces,number of tests with error or failure logged,unique errors or failures logged,number of possible liveness bugs,number of safety violations,mean execution time
0,b0,random,False,100,34.79,300,0,{},6,0,0 days 00:00:16.745741770
1,b0,random,True,100,28.28,300,2,{E0824},13,0,0 days 00:00:17.191952710
2,b0,pos,False,100,35.38,300,2,{E0824},1,0,0 days 00:00:17.228563320
3,b0,pos,True,100,29.02,300,1,{E0824},12,0,0 days 00:00:17.727142940
4,b0,pct2,False,67,4.07,168,0,{},95,0,0 days 00:00:17.191525940
5,b0,pct2,True,84,3.85,212,0,{},98,0,0 days 00:00:17.190402410
6,b0,posc,False,100,33.93,300,4,{E0824},7,0,0 days 00:00:17.048977010
7,b0,posc,True,100,28.59,300,3,{E0824},14,0,0 days 00:00:17.170510300
8,b0,pct1,False,52,2.89,115,1,{E0824},94,0,0 days 00:00:17.027642740
9,b0,pct1,True,73,3.27,170,0,{},98,0,0 days 00:00:16.985119440


Errors logged :

    E0824: 0 /src/braft_builder/brpc/src/bthread/mutex.cpp:497] bthread is suspended while holding1 pthread locks.

    E0826: /src/braft_builder/braft/src/braft/replicator.cpp:473] Group Counter fail, response term 3 mismatch, expect term 2

Failures logged : 

    F0826: Check failed: new_term >= _term (2 vs 3). 
    

In [13]:
tracesOfN2

0     [{'drop': 0, 'sender': 2, 'receiver': 1, 'acti...
1                                                    []
2     [{'drop': 1, 'sender': 1, 'receiver': 2, 'acti...
3     [{'drop': 1, 'sender': 0, 'receiver': 2, 'acti...
4     [{'drop': 0, 'sender': 0, 'receiver': 2, 'acti...
                            ...                        
95    [{'drop': 0, 'sender': 0, 'receiver': 2, 'acti...
96    [{'drop': 0, 'sender': 0, 'receiver': 2, 'acti...
97                                                   []
98    [{'drop': 0, 'sender': 1, 'receiver': 2, 'acti...
99    [{'drop': 0, 'sender': 2, 'receiver': 1, 'acti...
Name: Schedule, Length: 100, dtype: object